# FEATURE TRANSFORMATION

## IMPORT PACKAGES

In [2]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

#Automcomplete
%config IPCompleter.greedy=True

## IMPORT LOS DATA

1.- Project root

In [4]:
root = '/content/drive/MyDrive/02_RETAIL'

2.- Data files name

In [5]:
name_cat = 'cat_result_eda.pickle'
name_num = 'num_result_eda.pickle'

3.- Load data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cat = pd.read_pickle(root + '/02_Data/03_Work/' + name_cat)
num = pd.read_pickle(root + '/02_Data/03_Work/' + name_num)

## NEW VARIABLES

The dataset already incorporates:

* date component
* calendar variables

We'll create:

* the ones we identified on EDA
* lags
* mobile windows

We'll need to concat one dataframe again.

In [9]:
df = pd.concat([cat,num], axis = 1)
df

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,sales,sell_price
date,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25
2013-01-01,CA_3,FOODS_3_120,d_704,2013,1,4,Tuesday,NewYear,National,11249,33,1.25
2013-01-01,CA_3,FOODS_3_202,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_252,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_288,d_704,2013,1,4,Tuesday,NewYear,National,11249,20,4.28
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,FOODS_3_329,d_1767,2015,11,3,Monday,no_event,no_event,11544,9,1.68
2015-11-30,CA_4,FOODS_3_555,d_1767,2015,11,3,Monday,no_event,no_event,11544,26,2.48
2015-11-30,CA_4,FOODS_3_586,d_1767,2015,11,3,Monday,no_event,no_event,11544,13,2.48


### Intermittent variable demand

This variable will identify how many consecutive days have elapsed with zero sales.

We will define it as if the last n days have had zero sales then there is a stock break.

We can create several by changing the n.

It will help us to model.

In [10]:
def stock_break(sales, n = 5):
    zero_sales = pd.Series(np.where(sales == 0,1,0))
    num_zeros = zero_sales.rolling(n).sum()
    stock_break = np.where(num_zeros == n,1,0)
    return(stock_break)

In [11]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [12]:
df['stock_break_3'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,3)).values

In [13]:
df['stock_break_7'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,7)).values

In [14]:
df['stock_break_15'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,15)).values

### Lag variables

We are going to create lags on the following variables:

* sales: lags of 15 days
* sell_price: 7 day lags
* break_stock: lag of one day

In [15]:
def create_lags(df, variable, num_lags = 7):

    #Create the dataframe object
    lags = pd.DataFrame()

    #Crea every lags
    for each in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(each)] = df[variable].shift(each)

    #Return lags dataframe
    return(lags)

In [16]:
lags_sell_price_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(df = x, variable = 'sell_price', num_lags= 7))

<ipython-input-16-421d4b5bf552>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: create_lags(df = x, variable = 'sell_price', num_lags= 7))


In [17]:
lags_stock_break_3_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(df = x, variable = 'stock_break_3', num_lags= 1))

<ipython-input-17-dad288d0ba40>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: create_lags(df = x, variable = 'stock_break_3', num_lags= 1))


In [18]:
lags_stock_break_7_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(df = x, variable = 'stock_break_7', num_lags= 1))

<ipython-input-18-fdf5922f765c>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: create_lags(df = x, variable = 'stock_break_7', num_lags= 1))


In [19]:
lags_stock_break_15_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(df = x, variable = 'stock_break_15', num_lags= 1))

<ipython-input-19-b1dc44e6ac64>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: create_lags(df = x, variable = 'stock_break_15', num_lags= 1))


In [20]:
lags_sales_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: create_lags(df = x, variable = 'sales', num_lags= 15))

<ipython-input-20-9bc8c60a69d9>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: create_lags(df = x, variable = 'sales', num_lags= 15))


### Variables from mobile windows

We are going to create three types of mobile windows about sales:

* mobile minimum
* mobile average
* mobile maximum

Each of them in the range of 15 days.

In [21]:
def min_mobile(df, variable, num_periods = 7):

    minm = pd.DataFrame()

    for each in range(2,num_periods+1):
        minm[variable + '_minm_' + str(each)] = df[variable].shift(1).rolling(each).min()

    #Returns lags dataframe
    return(minm)

In [22]:
def mean_mobile(df, variable, num_periods = 7):

    mm = pd.DataFrame()

    for each in range(2,num_periods+1):
        mm[variable + '_mm_' + str(each)] = df[variable].shift(1).rolling(each).mean()

    #Returns lags dataframe
    return(mm)

In [23]:
def max_mobile(df, variable, num_periods = 7):

    maxm = pd.DataFrame()

    for each in range(2,num_periods+1):
        maxm[variable + '_maxm_' + str(each)] = df[variable].shift(1).rolling(each).max()

    #Returns lags dataframe
    return(maxm)

In [24]:
min_mobile_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: min_mobile(df = x, variable = 'sales', num_periods= 15))

<ipython-input-24-5f4dff7b4ee1>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: min_mobile(df = x, variable = 'sales', num_periods= 15))


In [25]:
mean_mobile_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: mean_mobile(df = x, variable = 'sales', num_periods= 15))

<ipython-input-25-642349d18b6b>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: mean_mobile(df = x, variable = 'sales', num_periods= 15))


In [26]:
max_mobile_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: max_mobile(df = x, variable = 'sales', num_periods= 15))

<ipython-input-26-e37aaab38c01>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: max_mobile(df = x, variable = 'sales', num_periods= 15))


## PREPARE THE DATASETS

### Join all generated dataframes

In [27]:
df_joined = pd.concat([df,
                      lags_sell_price_df,
                      lags_stock_break_3_df,
                      lags_stock_break_7_df,
                      lags_stock_break_15_df,
                      lags_sales_df,
                      min_mobile_df,
                      mean_mobile_df,
                      max_mobile_df], axis = 1)

df_joined

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,...,sales_maxm_6,sales_maxm_7,sales_maxm_8,sales_maxm_9,sales_maxm_10,sales_maxm_11,sales_maxm_12,sales_maxm_13,sales_maxm_14,sales_maxm_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,no_event,no_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,no_event,no_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,no_event,no_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,no_event,no_event,11250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,CA_4,FOODS_3_714,d_1764,2015,11,7,Friday,no_event,no_event,11543,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
2015-11-28,CA_4,FOODS_3_714,d_1765,2015,11,1,Saturday,no_event,no_event,11544,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


### Remove the nulls that have generated the new variables

In [28]:
df_joined.dropna(inplace=True)

### Eliminate the variables that we are not going to need to model

In [29]:
to_eliminate = ['d','wm_yr_wk','sell_price','stock_break_3','stock_break_7','stock_break_15']

In [30]:
df_joined.drop(columns=to_eliminate, inplace=True)

### Identify the target

In [31]:
target = df_joined.sales

### Separate num and cat

In [32]:
cat = df_joined.select_dtypes(include='O')

In [33]:
num = df_joined.select_dtypes(exclude='O')

## CATEGORICAL TRANSFORMATION

### One Hot Encoding

#### OHE variables

In [34]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instantiate

In [35]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Train and apply

In [36]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### Save as dataframe

In [37]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

#### TE variables

In [38]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instantiate

In [39]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Train and apply

In [40]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Save as dataframe

In [41]:
#Add suffixes to the names
names_te = [variable + '_te' for variable in var_te]

#Saves as dataframe
cat_te = pd.DataFrame(cat_te, columns = names_te)

## UNIFY TRANSFORMED DATASETS

### List all generated dataframes

We grab the segmentation variables from the df_joined.

In [42]:
from_df_joined = df_joined[['store_id','item_id']].reset_index()

from_df_joined.head(2)

,date,store_id,item_id
0,2013-01-16,CA_3,FOODS_3_090
1,2013-01-17,CA_3,FOODS_3_090


### Join all dataframes

In [43]:
dataframes = [from_df_joined, cat_ohe,cat_te,num.reset_index(drop=True)]

In [44]:
df_analytical_board = pd.concat(dataframes, axis = 1)

df_analytical_board

,date,store_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,sales_maxm_6,sales_maxm_7,sales_maxm_8,sales_maxm_9,sales_maxm_10,sales_maxm_11,sales_maxm_12,sales_maxm_13,sales_maxm_14,sales_maxm_15
0,2013-01-16,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
1,2013-01-17,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
2,2013-01-18,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
3,2013-01-19,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,281.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
4,2013-01-20,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20975,2015-11-26,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
20976,2015-11-27,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20977,2015-11-28,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20978,2015-11-29,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


## SAVE DATASET

In pickle format so as not to lose metadata modifications.

In [45]:
#Define file names
root_df_analytical_board = root + '/02_Data/03_Work/' + 'df_analytical_board.pickle'

In [46]:
#Guardar los archivos
df_analytical_board.to_pickle(root_df_analytical_board)